# Software Development Capability Analysis
## by Marc Vitalis

## Preliminary Wrangling

> Briefly introduce your dataset here.

In [ ]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from pandas.api.types import CategoricalDtype

%matplotlib inline

> Load in your dataset and describe its properties through the questions below.
Try and motivate your exploration goals through this section.

In [ ]:
workitems = pd.read_csv('workitems_master.csv')
workitems.head()

In [ ]:
workitems.info()

**Convert Dates to Date type**

Date format are represented as string (object), we should change them first to datetime format.

In [ ]:
workitems.new = pd.to_datetime(workitems.new)
workitems.doing = pd.to_datetime(workitems.doing)
workitems.done = pd.to_datetime(workitems.done)

workitems.head()

In [ ]:
workitems.info()

**Convert Releases to Numeric**

In [ ]:
workitems.loc[workitems.sprint.isna(), 'sprint'] = 0
workitems.sprint = workitems.sprint.astype(int)
workitems.info()

**Convert Category Types**

In [ ]:
workitems.workitem_type.value_counts()

In [ ]:
workitem_types = CategoricalDtype(categories = ['Story', 'Bug', 'Issue'], ordered=False)
workitems.workitem_type = workitems.workitem_type.astype(workitem_types)
workitems.head()

In [ ]:
workitems.describe()

### What is the structure of your dataset?

> The dataset consists of 2393, with 10 features (workitem_type, estimate, words, rel (release), sprint, assigned_to, new (date started), doing (date started working), done, and actual work (done - doing). Variables main point of interest are the date stamps for the work. Some are just to describe the work item such as sprint, release and assigned_to.

### What is/are the main feature(s) of interest in your dataset?

> I'm more interested how variables affects `actual_work`. The goal is find out for the patterns that affects the actual work.

### What features in the dataset do you think will help support your investigation into your feature(s) of interest?

> The dataset contains data and underwent to three (3) SDLC pattern (non-structured, semi-agile, scrum). The date stamps are very important (`new`, `doing`, `done`), this will help me extract important information, such as days of the week, months, or observe the time flow pattern if the SDLC pattern improves through time, or made it worst. As bonus I can also make use the correlation of titles to the actual work.

## Univariate Exploration

> First to explore is the main point of interest, `actual_work`.

In [ ]:
binsize = 2
bins = np.arange(0, workitems.actual_work.max()+binsize, binsize)

plt.figure(figsize=[8, 5])
plt.hist(data = workitems, x = 'actual_work', bins = bins)
plt.xlabel('Actual Work (Days)')
plt.show()

We have a long tail on the right, but most data are clumped between 1 to 20. We'll try log scale to spread out these clump a bit.

In [ ]:
# try log scale since it has a long tail
log_binsize = 0.1
bins = 10 ** np.arange(0, np.log10(workitems['actual_work'].max())+log_binsize, log_binsize)

plt.figure(figsize=[8, 5])
plt.hist(data = workitems, x = 'actual_work', bins = bins)
plt.xscale('log')
plt.xticks([2, 5, 8, 13, 20, 40, 50, 100, 200], [2, 5, 8, 13, 20, 40, 50, 100, 200])
plt.xlabel('Actual Work (Days)')
plt.show()

The graph looks trimodal, with peaks on 20, 5 and large numbers on 1. Let's examine what the averages tells us.

In [ ]:
[workitems.actual_work.mean(), workitems.actual_work.median(), workitems.actual_work.mode()]

We have different data here. Mean, says a workitem can be done 9 days, average. This is hardly conclusive as we have work items that's pulling the value up. Median however, makes a bit more sense as a normal workitem is normally done 3 days. 1 has more occurrence, but since this data is not categorical, it just provides a bit of information about the data.

Based from the information gathered from the team, one day of work are mostly possible for what they call `Bug Fest`. More exploring on the relationship of the `actual_work` and `workitem_type`. For now, let's investigate the distribution of the `workitem_type`.

In [ ]:
base_color = sb.color_palette()[0]
sb.countplot(data = workitems, x = 'workitem_type', color = base_color)

That's a big number of bugs in comparison to stories. Which makes sense for the spike in 1 day `actual_work` in the data. Let's explore the data further by looking into number of workitems being worked on per month.

In [ ]:
work_month = workitems.sort_values(['doing'])
work_month['month'] = work_month.doing.dt.strftime('%b %Y')

plt.figure(figsize=(10, 15))
sb.countplot(data = work_month, y = 'month', color = base_color)

As the date has become more recent, it's getting more consistent on the number of workitems. We don't know yet the distribution of how many of these are stories, bugs or issues. More on that later.

With scrums, there's a normal trend that during Fridays, there's a spike of things suddenly getting done. Let's investigate the distribution on weekdays, both on `doing` and `done`.

In [ ]:
weekdays_type = CategoricalDtype(categories=['Monday' , 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], ordered=True)

work_wk_doing = workitems

work_wk_doing['week'] = workitems.doing.dt.strftime('%A').astype(weekdays_type)

sb.countplot(data = work_wk_doing, x = 'week', color = base_color)

In [ ]:
work_wk_done = workitems

work_wk_done['week'] = workitems.done.dt.strftime('%A').astype(weekdays_type)

sb.countplot(data = work_wk_done, x = 'week', color = base_color)

This shows that the team is more productive when fresh during `Mondays` and tends to slow down throughout the week.

Let's observe the workitems per sprint.

In [ ]:
#TODO remove nan in sprint and change to int
work_sprint = workitems
work_sprint.loc[work_sprint.rel.isna(), 'rel'] = '0'
work_sprint = work_sprint.sort_values(['rel', 'sprint'])

work_sprint['rel_sprint'] = work_sprint.rel + '/' + work_sprint.sprint.astype(str).str.pad(width = 3, side = 'left', fillchar = '0')

plt.figure(figsize=(10, 15))
sb.countplot(data = work_sprint, y = 'rel_sprint', color = base_color)

The work items in each releases vary. Although there are items that slotted in Sprint zero which means there might be error in input on those.

Next to investigate is the estimates provided.

In [ ]:
binsize = 4
bins = np.arange(0, workitems.estimate.max()+binsize, binsize)

plt.figure(figsize=[8, 5])
plt.hist(data = workitems, x = 'estimate', bins = bins)
plt.xlabel('Points')
plt.show()

Again, it has a long tail. Let's try plotting the log.

In [ ]:
# try log scale since it has a long tail
log_binsize = 0.1
bins = 10 ** np.arange(0, np.log10(workitems['estimate'].max())+log_binsize, log_binsize)

plt.figure(figsize=[8, 5])
plt.hist(data = workitems, x = 'estimate', bins = bins)
plt.xscale('log')
plt.xticks([2, 5, 8, 13, 20, 40, 50, 100, 200], [2, 5, 8, 13, 20, 40, 50, 100, 200])
plt.xlabel('points')
plt.show()

Distribution has its peak between 8 to 13 estimates. Based on my conversation with the team, this is the 'just enough' size of stories.

Last to explore are the title broken down into words. Let's find out the top 40 word occurence.

In [ ]:
work_words = workitems[['id', 'words']]

work_words = work_words[['id', 'words']].words.str.split(',').apply(pd.Series) \
    .merge(work_words[['id', 'words']], right_index = True, left_index = True) \
    .drop(["words"], axis = 1) \
    .melt(id_vars = ['id'], value_name = "word") \
    .drop("variable", axis = 1) \
    .dropna()

plt.figure(figsize=(10, 15))
sb.countplot(data = work_words, y = 'word', color = base_color, order = work_words.word.value_counts().iloc[:40].index)

Results is interesting. Top entry is error, which probably evident mostly on bug items. Other items in top 40 mostly can describe the application itself.

### Discuss the distribution(s) of your variable(s) of interest. Were there any unusual points? Did you need to perform any transformations?

> Your answer here!

### Of the features you investigated, were there any unusual distributions? Did you perform any operations on the data to tidy, adjust, or change the form of the data? If so, why did you do this?

> Your answer here!

## Bivariate Exploration

> In this section, investigate relationships between pairs of variables in your
data. Make sure the variables that you cover here have been introduced in some
fashion in the previous section (univariate exploration).

### Talk about some of the relationships you observed in this part of the investigation. How did the feature(s) of interest vary with other features in the dataset?

> Your answer here!

### Did you observe any interesting relationships between the other features (not the main feature(s) of interest)?

> Your answer here!

## Multivariate Exploration

> Create plots of three or more variables to investigate your data even
further. Make sure that your investigations are justified, and follow from
your work in the previous sections.

### Talk about some of the relationships you observed in this part of the investigation. Were there features that strengthened each other in terms of looking at your feature(s) of interest?

> Your answer here!

### Were there any interesting or surprising interactions between features?

> Your answer here!

> At the end of your report, make sure that you export the notebook as an
html file from the `File > Download as... > HTML` menu. Make sure you keep
track of where the exported file goes, so you can put it in the same folder
as this notebook for project submission. Also, make sure you remove all of
the quote-formatted guide notes like this one before you finish your report!